In [3]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [4]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Making `Rule of 25` for margins
- [ ] Get undPrices with IVs
- [ ] Get the option chains
- [ ] Compute 25% of undPrice * lots
- [ ] Make `dte curve`. Lower the dte, lower the margin penalty.
- [ ] Make the `strike curve`. Closer the strike to undPrice, lower the penatly.
- [ ] Compute `computedmgn` and populate df_chains

In [5]:
# imports
import asyncio

from utils import (get_file_age, get_pickle, get_prices_with_ivs, pickle_me, get_a_price_iv,
                   update_chains_dte, chunk_me, to_list, clean_ib_util_df, compute_sdev_right)

## Getting underlying prices

In [6]:
# Get und_prices
unds = get_pickle(DATAPATH / 'unds.pkl')
df_unds = pd.DataFrame.from_dict(unds.items())
df_unds.columns = ['symbol', 'contract']

In [5]:
def split_symbol_price_iv(prices_dict: dict) -> pd.DataFrame:

    """Splits symbol, prices and ivs into a df.

    To be used after get_mkt_prices()"""

    symbols = []
    prices = []
    ivs = []

    for symbol, v in prices_dict.items():
        price, iv = v
        
        symbols.append(symbol)
        prices.append(price)
        ivs.append(iv)
        
    df_prices = pd.DataFrame({'symbol': symbols, 'price': prices, 'iv': ivs })

    return df_prices

In [6]:
from tqdm.asyncio import tqdm

contracts = df_unds.contract
chunk_size = 44
sleep = 7  # import time

async def get_mkt_prices(port: int, 
                         contracts: list, 
                         chunk_size: int=44, 
                         sleep: int=7) -> pd.DataFrame:

    contracts = to_list(contracts)
    chunks = tqdm(chunk_me(contracts, chunk_size), desc="Getting market prices with IVs")
    results = dict()
    with await IB().connectAsync(port=port) as ib:
        for cts in chunks:
            tasks = [asyncio.create_task(get_a_price_iv(ib, c, sleep)) for c in cts]
            res = await asyncio.gather(*tasks)

            for r in res:
                symbol, price, iv = r
                results[symbol] = (price, iv)

    df_prices = split_symbol_price_iv(results)
    df_prices = pd.merge(clean_ib_util_df(contracts).iloc[:, :6], df_prices, on='symbol')

    # remove unnecessary columns (for secType == `STK`)
    keep_cols = ~((df_prices == 0).all() | \
              (df_prices == "").all() | \
                df_prices.isnull().all())
    
    df_prices = df_prices.loc[:, keep_cols[keep_cols == True].index]

    return df_prices

In [7]:
df_prices = asyncio.run(get_mkt_prices(port=PORT, contracts=df_unds.contract))

Getting market prices with IVs:   0%|          | 0/6 [00:00<?, ?it/s]

2024-03-02 16:17:47.253 | INFO     | utils:get_a_price_iv:879 - No price found for COF!
Getting market prices with IVs: 100%|██████████| 6/6 [02:03<00:00, 20.56s/it]


## Getting the `chains`

In [10]:
pickled_chains = DATAPATH / 'df_chains.pkl'
MINUTES_GAP = 15 # Ensures that the files are updated at least 4 times an hour!

# check age of `df_chains.pkl`
pickle_chains_age = get_file_age(pickled_chains)

# get the pickled chains and remove dte <=0
df_chains = get_pickle(pickled_chains)

td_in_minutes = pickle_chains_age.td.total_seconds()/60

if td_in_minutes > MINUTES_GAP:
    df_chains = update_chains_dte(df_chains, MARKET)
    df = compute_sdev_right(df_chains)
    pickle_me(df, pickled_chains)

### Calculate `sdev` and `right`

In [1]:
from utils import get_a_stdev

In [7]:
def compute_strike_sd_right(df: pd.DataFrame) -> pd.DataFrame:
    """Computes strike's standard deviation and right for option chains
    
    Note:
    ----
    Function needs `iv`, `undprice` and `dte` columns in df"""

    # remove dtes <= 0 to prevent math failure
    df = df[df.dte > 0].reset_index(drop=True)

    # compute strike's standard deviation
    df = df.assign(sigma=df[['iv', 'undPrice', 'dte']].\
                    apply(lambda x: get_a_stdev(x.iv, x.undPrice, x.dte), axis=1))

    df = df.assign(strike_sdev = (df.strike - df.undPrice) / df.sigma)

    # determine the right
    df = df.assign(right = df.strike_sdev.apply(lambda strike_sdev: 'P' if strike_sdev < 0 else 'C'))

    return df

In [8]:
def target_options_with_adjusted_sdev(df_chains: pd.DataFrame,
                                      STDMULT: float,
                                      how_many: int,
                                      DTESTDEVLOW: float, 
                                      DTESTDEVHI: float,
                                      DECAYRATE: float,
                                      MARKET_IS_OPEN: bool) -> pd.DataFrame:
    
    """Adjust the standard deviation to DTE, penalizes DTES closer to zero"""

    # Get the extra SD adjusted to DTE
    # xtra_sd = 1-(df_chains.dte/100)

    MARKET = df_chains.exchange.unique()[0]

    _vars = Vars(MARKET)

    # Factor a bump to dev if market is not open
    if MARKET_IS_OPEN:
        GAPBUMP = 0
    else:
        GAPBUMP = _vars.GAPBUMP

    xtra_sd = df_chains.dte.apply(lambda dte: sdev_for_dte(dte=dte,
                                                           DTESTDEVLOW=DTESTDEVLOW, 
                                                           DTESTDEVHI=DTESTDEVHI,
                                                           DECAYRATE=DECAYRATE,
                                                           GAPBUMP=GAPBUMP
                                                           ))

    # Build the series for revised SD
    sd_revised = STDMULT + xtra_sd if STDMULT > 0 else STDMULT - xtra_sd

    # Identify the closest standerd devs to the revised SD\
    df_ch = df_chains.assign(sd_revised=sd_revised)
    closest_sdevs = df_ch.groupby(['symbol', 'dte'])[['sdev', 'sd_revised']]\
        .apply(lambda x: get_closest_values(x.sdev, 
                                            x.sd_revised.min(), 
                                            how_many))
    closest_sdevs.name = 'sdev1' 

    # Join the closest chains to the closest revised SD
    df_ch1 = df_ch.set_index(['symbol', 'dte']).join(closest_sdevs)

    # Get the target chains
    df_ch2 = df_ch1[df_ch1.apply(lambda x: x.sdev in x.sdev1, axis=1)] \
                        .reset_index()
    
    return df_ch2